In [1]:
import os
import sys
from pathlib import Path, PurePath as PPath
from dotenv import load_dotenv, find_dotenv

from datetime import datetime
from collections import OrderedDict, defaultdict

#print('Python ver: {}\nPython env: {}'.format(sys.version, Path(sys.prefix).name))
#print('Currrent dir: {}\n'.format(Path.cwd()))

def add_to_sys_path(this_path, up=False):
    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """
    if up:
        # NB: Path does not have a str method.
        newp = str(PPath(this_path).parent)
    else:
        newp = str(PPath(this_path)) 
    
    if newp not in sys.path:
        sys.path.insert(1, newp)
        #print('Path added to sys.path: {}'.format(newp))

# if notebook inside another folder, eg ./notebooks:
nb_folder = 'notebooks'
add_to_sys_path(Path.cwd(), Path.cwd().name.startswith(nb_folder))


def get_project_dirs(which=['data', 'images'], nb_folder='notebooks'):
    dir_lst = []
    if Path.cwd().name.startswith(nb_folder):
        dir_fn = Path.cwd().parent.joinpath
    else:
        dir_fn = Path.cwd().joinpath
        
    for d in which:
        DIR = dir_fn(d)
        if not DIR.exists():
            Path.mkdir(DIR)
        dir_lst.append(DIR)
    return dir_lst

DIR_DATA, DIR_IMG = get_project_dirs()

DATA_SHAPEFILES = DIR_DATA.joinpath('shapefiles')
DATA_RAW = DIR_DATA.joinpath('raw')
DATA_INTERIM = DIR_DATA.joinpath('intermediate')
DATA_READY = DIR_DATA.joinpath('production')

def data_subfolders():
    ans = int(input('Setup data subfolders?  1=yes, 0=no: '))
    if ans == 1:
        for d in [DATA_RAW, DATA_INTERIM, DATA_READY, DATA_SHAPEFILES]:
            if not d.exists():
                Path.mkdir(d)
                print(f'Created: {d}')
                
import numpy as np
import scipy as sp
from scipy import stats as sps
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option("display.max_colwidth", 300)

import json

import matplotlib as mpl
from matplotlib import pyplot as plt
plt.ion()
# color cycler too short (< weekdays) w/ seaborn 
plt.style.use(['tableau-colorblind10', 'fast'])

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, Image, HTML, Markdown #, IFrame

# autoreload extension
from IPython import get_ipython
ipython = get_ipython()

if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

In [2]:
# local utils lib
import etl
from display_figure import display_figure

Questions = {'Q1': "Has the arrest rate been decreasing from 2015-2018?",
             'Q2': "What are the top 5 most frequent arrests as described in the column 'pd_desc' in 2018?",
             'Q3': "If we think of arrests as a sample of total crime, is there more crime in <br>precinct 19 (Upper East Side) than precinct 73 (Brownsville)?",
             'Q4': "Given the available data, what model would you build to predict crime<br>to better allocate NYPD resources?"
            }

ID_Arrests = '8h9b-rp9u'
raw_meta = DATA_RAW.joinpath(ID_Arrests + '_Audit_medadata.json')

# number of arrests DESC properly categorized with LAW_CAT over 2015-2018 period:
n_LAW_CAT_DESC = 350

# for figures
style_dict = {'div': {'width':80},
              'figure': {'display':'inline-block', 'text-align':'left'},
              'image': {'display':'block', 'width': 500, 'height':400},
              'caption': {'color':'Navy','font-size': 'small',
                          'font-family': 'Arial, Helvetica, sans-serif'}
             }

with open(raw_meta) as fh:
    meta_Arrests = json.load(fh)

# Dataset full name, Total row count (raw data):
dataset_name = meta_Arrests['name']
dataset_rows = etl.get_rows_from_metadata(meta_Arrests, ID_Arrests)

#clean dataset
data_rows = 4798119

In [3]:
id_url = "https://data.cityofnewyork.us/Public-Safety/NYPD-Arrests-Data-Historic-/8h9b-rp9u/"
fig2 = DIR_IMG.joinpath(ID_Arrests + '_Top5_2018_evol.png')

# all yrs:
top5 = DATA_RAW.joinpath(ID_Arrests + '_top5_desc.csv')
top5_2018 = DATA_INTERIM.joinpath(ID_Arrests + '_top5_desc_2018.csv')
top5_2017 = DATA_INTERIM.joinpath(ID_Arrests + '_top5_desc_2017.csv')
top5_2016 = DATA_INTERIM.joinpath(ID_Arrests + '_top5_desc_2016.csv')
top5_2015 = DATA_INTERIM.joinpath(ID_Arrests + '_top5_desc_2015.csv')

top5_evol_final = DATA_READY.joinpath(ID_Arrests + '_top5_desc_evol.csv')
top5_evol_rankings = DATA_READY.joinpath(ID_Arrests + '_top5_desc_evol_rankings.csv')

df = etl.load_xtab_df(top5_2018, colindex_name=2018)
df_evol = etl.load_xtab_df(top5_evol_final)
evol_stats = etl.get_evol_stats(top5_evol_final)
df_top5_ranks = etl.load_xtab_df(top5_evol_rankings)
df_top5_ranks.replace(np.nan, '', inplace=True)
n_common_desc = df_top5_ranks.shape[0]


cap2 = f'<i><b>{dataset_name}</b> (<a href="{id_url}">{ID_Arrests}</a>)</i>: '
cap2 += 'Evolution of the 2018 Top 5 arrests over the period (2015 - 2018).<br>' 
caption2_dict = {'number': 2, 'caption': cap2}
fig2_html = display_figure(fig2, style_dict, caption2_dict, return_html=True)

---
# NYC Arrests  - Report 

#### Reporter: Cat Chenal  
#### Email: catchenal@gmail.com
#### [GitHub](https://github.com/CatChenal/NYCData)
---

In [4]:
H = '<div style="width:75%;border-color:DodgerBlue;background-color:#e6ffff;padding:30px;">'
text = H + '<h3>Summary:</h3>'
text += "Even though the number of arrests has steadily dropped in the 2015-2018 period (~ 30% per year), there is a remarkable constance in key factors: <br>"
text += "<p>&nbsp;&nbsp;<b>a.</b> Type of arrests: only 8 types are common to all years' Top 5 ranking, while the dataset reports over 350 different violations of the NYS penal code.<br>"
text += "&nbsp;&nbsp;<b>b.</b> Criminality has its days! The lowest levels occur on Sundays, the highest on Wednesday.</p>"
text +=  "<h4>Dataset quality: <i>NYPD Arrests Data (Historic)</i></h4><p>Missing data were imputed if possible. "
text += "The clean dataset is reduced by 220 uncategorized arrests to 4,798,119, but There are still 8,650 records with an empty 'pd_desc' field.<br>"
text += "The top 3 Precincts with uncategorized arrests are: P14 (53), P103 (10) and P73 (6), amounting to 69 out of 220, or over 31%."
Markdown(text)

<div style="width:75%;border-color:DodgerBlue;background-color:#e6ffff;padding:30px;"><h3>Summary:</h3>Even though the number of arrests has steadily dropped in the 2015-2018 period (~ 30% per year), there is a remarkable constance in key factors: <br><p>&nbsp;&nbsp;<b>a.</b> Type of arrests: only 8 types are common to all years' Top 5 ranking, while the dataset reports over 350 different violations of the NYS penal code.<br>&nbsp;&nbsp;<b>b.</b> Criminality has its days! The lowest levels occur on Sundays, the highest on Wednesday.</p><h4>Dataset quality: <i>NYPD Arrests Data (Historic)</i></h4><p>Missing data were imputed if possible. The clean dataset is reduced by 220 uncategorized arrests to 4,798,119, but There are still 8,650 records with an empty 'pd_desc' field.<br>The top 3 Precincts with uncategorized arrests are: P14 (53), P103 (10) and P73 (6), amounting to 69 out of 220, or over 31%.

In [8]:
fig1 = DIR_IMG.joinpath(ID_Arrests + '_years_counts_from_2015.png')

cap1 = f'<i><b>{dataset_name}</b> (<a href="{id_url}">{ID_Arrests}</a>)</i>: '
cap1 += 'Yearly total number of arrests (2015 - 2018).<br>' 
caption1_dict = {'number': 1, 'caption': cap1}
fig1_html = display_figure(fig1, style_dict, caption1_dict, return_html=True)

ans = '<div style="{width:80%}"><h3>' + f'{Questions["Q1"]}</h3>'
ans += fig1_html
ans += "<p>&nbsp;&nbsp;Yes:  The percentage decrease in the number of NYPD arrests over the last four years (2015 - 2018) is 27%."
ans += "Additionally, the yearly totals have decreased at a steady pace over that period. As depicted in Figure 1, "
ans += "the linear change in the yearly totals amounts to a decrease of over 30,000 arrests per year.<br>"
ans += "(<i>Statistical note: the high coefficient of determination (R<sup>2</sup>) ascertains the data is properly modelled by a line.</i>)"

ans +='<h3>' + f'{Questions["Q2"]}</h3>'
ans +="<br><strong>These are the Top 5 arrests in 2018:</strong><br> " + df.to_html(index=True)
ans += "</p></div>"
Markdown(ans)

<div style="{width:80%}"><h3>Has the arrest rate been decreasing from 2015-2018?</h3><figure style="display:inline-block;text-align:left;">
      <img src="C:\Users\catch\Documents\GH_Projects\NYC_data\images\8h9b-rp9u_years_counts_from_2015.png" 
         alt="x"
         style="display:block;width:500;height:400;"
         title=""
      >
      <figcaption style="color:Navy;font-size:small;font-family:Arial, Helvetica, sans-serif;">
        Figure 1 - <i><b>NYPD Arrests Data (Historic)</b> (<a href="https://data.cityofnewyork.us/Public-Safety/NYPD-Arrests-Data-Historic-/8h9b-rp9u/">8h9b-rp9u</a>)</i>: Yearly total number of arrests (2015 - 2018).<br>
      </figcaption></figure>
    <p>&nbsp;&nbsp;Yes:  The percentage decrease in the number of NYPD arrests over the last four years (2015 - 2018) is 27%.Additionally, the yearly totals have decreased at a steady pace over that period. As depicted in Figure 1, the linear change in the yearly totals amounts to a decrease of over 30,000 arrests per year.<br>(<i>Statistical note: the high coefficient of determination (R<sup>2</sup>) ascertains the data is properly modelled by a line.</i>)<h3>What are the top 5 most frequent arrests as described in the column 'pd_desc' in 2018?</h3><br><strong>These are the Top 5 arrests in 2018:</strong><br> <table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>2018</th>
      <th>count</th>
    </tr>
    <tr>
      <th>pd_desc</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>ASSAULT 3</th>
      <td>26611</td>
    </tr>
    <tr>
      <th>LARCENY,PETIT FROM OPEN AREAS,UNCLASSIFIED</th>
      <td>23405</td>
    </tr>
    <tr>
      <th>TRAFFIC,UNCLASSIFIED MISDEMEAN</th>
      <td>14856</td>
    </tr>
    <tr>
      <th>ASSAULT 2,1,UNCLASSIFIED</th>
      <td>11763</td>
    </tr>
    <tr>
      <th>CONTROLLED SUBSTANCE, POSSESSION 7</th>
      <td>9982</td>
    </tr>
  </tbody>
</table></p></div>

In [6]:
ans = '<div style="{width:80%}"><p>When traced over the 2015-2018 period, most display a remarkable stability '
ans += "(measured as the percentage change ('%change') between the first and last year):</p>"
ans += df_evol.to_html(index=True)
ans += f"<p>In fact, there are only {evol_stats['count_delta_above_5pct']} types of arrest that changed by over 5%, "
ans += f"with '{evol_stats['min_delta_desc']}' having the largest drop ({evol_stats['min_delta']:.0%}). This persistence is clearly "
ans += "depicted in Figure 2:</p>" 
ans += fig2_html
ans += "<p><br>The persistence of a few number of arrest types with high count is also occurring when we consider the Top 5 arrests <br>"
ans += f"<i>in each year</i>:  there are {n_LAW_CAT_DESC} (properly categorized) arrests in the 2015-2018 period, but only "
ans += f"{n_common_desc} that appear among the Top 5 yearly rankings:</p></div>"

Markdown(ans)
etl.top5_style(df_top5_ranks)

xtab_2018 = DATA_INTERIM.joinpath(ID_Arrests + '_2018_YMD_xtab.csv')
MDxt_18 = etl.load_xtab_df(xtab_2018, colindex_name='Month')
MDxt_18_styl = etl.highlights_styled(MDxt_18)

ans = '<div style="{width:80%}"><p><h5>Total number of arrests per day and month in 2018</h5>'
ans += "Criminality has its days!  The lowest level occurs on Sundays, the highest on Wednesday "
ans += "(true for all years):</p><i>(Monthly min: green, max: orange.)</i></p></div>"
Markdown(ans)

MDxt_18_styl

<div style="{width:80%}"><p>When traced over the 2015-2018 period, most display a remarkable stability (measured as the percentage change ('%change') between the first and last year):</p><table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>2015</th>
      <th>2016</th>
      <th>2017</th>
      <th>2018</th>
      <th>%change</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>ASSAULT 3</th>
      <td>27631.0</td>
      <td>26961.0</td>
      <td>26281.0</td>
      <td>26611.0</td>
      <td>-0.04</td>
    </tr>
    <tr>
      <th>LARCENY,PETIT FROM OPEN AREAS,UNCLASSIFIED</th>
      <td>25772.0</td>
      <td>23871.0</td>
      <td>23020.0</td>
      <td>23405.0</td>
      <td>-0.09</td>
    </tr>
    <tr>
      <th>TRAFFIC,UNCLASSIFIED MISDEMEAN</th>
      <td>14435.0</td>
      <td>15231.0</td>
      <td>16298.0</td>
      <td>14856.0</td>
      <td>0.03</td>
    </tr>
    <tr>
      <th>ASSAULT 2,1,UNCLASSIFIED</th>
      <td>12048.0</td>
      <td>12127.0</td>
      <td>11911.0</td>
      <td>11763.0</td>
      <td>-0.02</td>
    </tr>
    <tr>
      <th>CONTROLLED SUBSTANCE, POSSESSION 7</th>
      <td>17420.0</td>
      <td>14177.0</td>
      <td>12341.0</td>
      <td>9982.0</td>
      <td>-0.43</td>
    </tr>
  </tbody>
</table><p>In fact, there are only 2 types of arrest that changed by over 5%, with 'CONTROLLED SUBSTANCE, POSSESSION 7' having the largest drop (-43%). This persistence is clearly depicted in Figure 2:</p><figure style="display:inline-block;text-align:left;">
      <img src="C:\Users\catch\Documents\GH_Projects\NYC_data\images\8h9b-rp9u_Top5_2018_evol.png" 
         alt="x"
         style="display:block;width:500;height:400;"
         title=""
      >
      <figcaption style="color:Navy;font-size:small;font-family:Arial, Helvetica, sans-serif;">
        Figure 2 - <i><b>NYPD Arrests Data (Historic)</b> (<a href="https://data.cityofnewyork.us/Public-Safety/NYPD-Arrests-Data-Historic-/8h9b-rp9u/">8h9b-rp9u</a>)</i>: Evolution of the 2018 Top 5 arrests over the period (2015 - 2018).<br>
      </figcaption></figure>
    <p><br>The persistence of a few number of arrest types with high count is also occurring when we consider the Top 5 arrests <br><i>in each year</i>:  there are 350 (properly categorized) arrests in the 2015-2018 period, but only 8 that appear among the Top 5 yearly rankings:</p></div>

,2015,2016,2017,2018
Arrests description in any yearly Top 5 ranking,,,,
ASSAULT 3,"2nd: 27,631","2nd: 26,961","1st: 26,281","1st: 26,611"
"LARCENY,PETIT FROM OPEN AREAS,UNCLASSIFIED","3rd: 25,772","3rd: 23,871","2nd: 23,020","2nd: 23,405"
"TRAFFIC,UNCLASSIFIED MISDEMEAN",,"5th: 15,231","5th: 16,298","3rd: 14,856"
"ASSAULT 2,1,UNCLASSIFIED",,,,"4th: 11,763"
"CONTROLLED SUBSTANCE, POSSESSION 7","4th: 17,420",,,"5th: 9,982"
"THEFT OF SERVICES, UNCLASSIFIED","1st: 29,833","1st: 27,234","3rd: 19,825",
"MARIJUANA, POSSESSION 4 & 5",,"4th: 18,117","4th: 17,964",
"NY STATE LAWS,UNCLASSIFIED VIOLATION","5th: 16,899",,,


<div style="{width:80%}"><p><h5>Total number of arrests per day and month in 2018</h5>Criminality has its days!  The lowest level occurs on Sundays, the highest on Wednesday (true for all years):</p><i>(Monthly min: green, max: orange.)</i></p></div>

Month,1,2,3,4,5,6,7,8,9,10,11,12
Day,,,,,,,,,,,,
Sun,2203,2043,2263,2554,2061,1826,2619,2206,2293,1858,1859,2103
Mon,3273,2687,2593,2875,2300,2275,2976,2319,2115,2753,2131,2280
Tue,4381,3530,3467,3201,3746,2896,3642,2909,2608,3555,2634,2356
Wed,4764,3668,2795,3484,4293,3169,3026,4066,3017,3751,2872,2687
Thu,3261,3913,4268,3479,4031,3079,3144,3834,2884,2924,3260,2652
Fri,3211,3137,3727,3051,3193,3382,2899,3619,2732,2642,3144,2483
Sat,2789,2746,3349,2683,2412,2985,2569,2427,2859,2024,2096,2834


---

In [7]:
nyc_pop = 8175133
# https://www1.nyc.gov/assets/planning/download/pdf/data-maps/nyc-population/census2010/pgrhc.pdf
ID_CD_pop = 'xi7c-iiu2'
pops = DATA_RAW.joinpath(ID_CD_pop + '_CD_pop_2010.csv')
df_pop = etl.load_xtab_df(pops)

# problem recovering mutiindex df from csv:
arrests_totals = DATA_READY.joinpath(ID_Arrests + '_precincts_arrests_tots.csv')
# saved as pic for report:
fig_arrests_tos = DIR_IMG.joinpath(ID_Arrests + '_arrests_totals.png')

tots = etl.load_arrests_totals(arrests_totals)

p19_tot_all_pct = tots.loc['19', 'all years'] / tots.loc['all', 'all years']
p19_tot_4y_pct = tots.loc['19', 'last 4'] / tots.loc['all', 'last 4']

p73_tot_all_pct = tots.loc['73', 'all years'] / tots.loc['all', 'all years']
p73_tot_4y_pct = tots.loc['73', 'last 4'] / tots.loc['all', 'last 4']

pop19cd = df_pop.loc[0,'pop_2010']
pop73cd = df_pop.loc[1,'pop_2010']

Q = '<div style="{width:75%}"><h3>' + f'{Questions["Q3"]}</h3>'
text = Q + f'<p>&nbsp;&nbsp;Precinct 19 accounts for less than 1% of all crimes and of those in the last'
text += f'four years ({p19_tot_all_pct:.2%} and {p19_tot_4y_pct:.2%}, respectively) while '
text += f'the percentages for Precinct 79 are {p73_tot_all_pct:.2%} and {p73_tot_4y_pct:.2%}, '
text += 'i.e. over twice as much. As the population of Community District 8, where Precinct 19 resides, ' 
text += 'is slightly over 2.5 larger than that of CD 16, where Precinct 73 is located, this is not a population effect: '
text += 'there is more crime in Precinct 73 than in Precinct 19.</p>'

text += f'<h3>{Questions["Q4"]}</h3>'
text += """<p>&nbsp;&nbsp;Predicitve crime modelling is an active area of research due to its data, design and ethical challenges. 
These spatio-temporal models have stringent requirements for their effectiveness: For the purpose of allocating patrol officers to
needed areas in a timely fashion, the model would need to predict the locations (within a reasonable distance variability) <i>together with</i>
a long enough "time-ahead" window. For instance a model capable of predicting a crime hot-spot at the block level would
be useless if it could not do so <i>at least</i> 48 hours prior the predicted activity. As the _NYPD Arrests Data_ dataset does not have
time data, other sources, such as 911 call logs, are needed. </p>
<p>&nbsp;&nbsp;Part of the data challenge is to decide which input data to use: while
weather and "crowd signals" from social media are probably useful, the use of socio-economic data may introduce bias.
&nbsp;&nbsp;The more varied the sources, the wider the skills needed to process and anlyze the data as they take many forms: text, sound,
video, etc. Part of the modeling challenge is to identify which features are necessary to achieve a given goal. In this context, the 
minimal type of information is location, date and time and the age-group distibutions (at the census tract level if possible) since
the 24-44 age group is the most represented in the dataset.  <br>
&nbsp;&nbsp;Whether the model is parametric (i.e. regression) or neural (machine learning), the first step is to establish a baseline against which
other models (issued from different feature selection, for example) will be rated.</p></div>"""

Markdown(text)

<div style="{width:75%}"><h3>If we think of arrests as a sample of total crime, is there more crime in <br>precinct 19 (Upper East Side) than precinct 73 (Brownsville)?</h3><p>&nbsp;&nbsp;Precinct 19 accounts for less than 1% of all crimes and of those in the lastfour years (0.85% and 0.92%, respectively) while the percentages for Precinct 79 are 2.79% and 2.40%, i.e. over twice as much. As the population of Community District 8, where Precinct 19 resides, is slightly over 2.5 larger than that of CD 16, where Precinct 73 is located, this is not a population effect: there is more crime in Precinct 73 than in Precinct 19.</p><h3>Given the available data, what model would you build to predict crime<br>to better allocate NYPD resources?</h3><p>&nbsp;&nbsp;Predicitve crime modelling is an active area of research due to its data, design and ethical challenges. 
These spatio-temporal models have stringent requirements for their effectiveness: For the purpose of allocating patrol officers to
needed areas in a timely fashion, the model would need to predict the locations (within a reasonable distance variability) <i>together with</i>
a long enough "time-ahead" window. For instance a model capable of predicting a crime hot-spot at the block level would
be useless if it could not do so <i>at least</i> 48 hours prior the predicted activity. As the _NYPD Arrests Data_ dataset does not have
time data, other sources, such as 911 call logs, are needed. </p>
<p>&nbsp;&nbsp;Part of the data challenge is to decide which input data to use: while
weather and "crowd signals" from social media are probably useful, the use of socio-economic data may introduce bias.
&nbsp;&nbsp;The more varied the sources, the wider the skills needed to process and anlyze the data as they take many forms: text, sound,
video, etc. Part of the modeling challenge is to identify which features are necessary to achieve a given goal. In this context, the 
minimal type of information is location, date and time and the age-group distibutions (at the census tract level if possible) since
the 24-44 age group is the most represented in the dataset.  <br>
&nbsp;&nbsp;Whether the model is parametric (i.e. regression) or neural (machine learning), the first step is to establish a baseline against which
other models (issued from different feature selection, for example) will be rated.</p></div>